# Memory usage measurements

In [4]:
import sys
import subprocess
import shutil
from pathlib import Path

assert shutil.which('valgrind')
assert shutil.which('ms_print')

CWD = Path.cwd()

def invoke_massif(args, output):
  subprocess.run([
    'valgrind', '--tool=massif', f'--massif-out-file={output}'
  ]  + args, cwd=CWD)


def memory_graph(test_name, code):
  massif_output = test_name + '.massif'
  cwd = Path().cwd()
  invoke_massif([
    sys.executable, '-c', ';'.join(code.split('\n'))
  ], massif_output)
  return '\n'.join(subprocess.check_output(
    ['ms_print', massif_output], cwd=CWD
  ).decode().split('\n\n')[:2])


In [5]:
random_matr = '''\
import numpy as np
n = 4096
m = 2 * n
matr = np.zeros((n, m), dtype=np.complex64)
matr = np.random.rand(n, m)
'''

Numpy:

In [6]:
numpy_test = random_matr + '''\
import numpy
expected = numpy.fft.fft2(matr)
print(expected)
'''

print(memory_graph('numpy', numpy_test))

==5346== Massif, a heap profiler
==5346== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==5346== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==5346== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex64);matr\ =\ np.random.rand(n,\ m);import\ numpy;expected\ =\ numpy.fft.fft2(matr);print(expected);
==5346== 


[[ 1.67752013e+07   +0.j          1.13545085e+03 -584.8092039j
  -1.00712503e+03   -4.65580131j ... -1.54517113e+03 -394.14066937j
  -1.00712503e+03   +4.65580131j  1.13545085e+03 +584.8092039j ]
 [-2.05320981e+03+1353.10348998j  6.32204464e+02+2156.4140635j
  -1.80167953e+03 +110.68879505j ...  1.78651405e+02-2652.73450877j
   1.78331957e+03-1140.92800491j -5.65434686e+02 +371.573199j  ]
 [ 1.48360034e+02 +850.61306796j  2.57573832e+03+1214.32915344j
  -5.80393310e+02  -29.81744381j ... -4.23733211e+02 -726.70470492j
   2.20572587e+03 +209.4879722j  -2.61257353e+02 -159.41538668j]
 ...
 [-5.16667698e+02+2425.85438804j  6.53719519e+02+1863.55740167j
   1.27312808e+03 +631.53274903j ... -1.19253243e+03-1647.51247303j
   8.08162514e+02+1809.70500012j  4.20751666e+01+1307.64767038j]
 [ 1.48360034e+02 -850.61306796j -2.61257353e+02 +159.41538668j
   2.20572587e+03 -209.4879722j  ...  7.56398529e+02-1309.40110345j
  -5.80393310e+02  +29.81744381j  2.57573832e+03-1214.32915344j]
 [-2.0532098

==5346== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


FFT with type conversion:

In [7]:
fft_type_conversion = random_matr + '''\
import fft_project
expected = fft_project.fft2d(matr, return_copy=True, use_threads=False)
print(expected)
'''

print(memory_graph('type_conversions', fft_type_conversion))

==5474== Massif, a heap profiler
==5474== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==5474== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==5474== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex64);matr\ =\ np.random.rand(n,\ m);import\ fft_project;expected\ =\ fft_project.fft2d(matr,\ return_copy=True,\ use_threads=False);print(expected);
==5474== 


[[ 1.67784815e+07   +0.j         -2.16221235e+03-1422.69809849j
  -2.29146292e+03 -869.8489139j  ...  2.01126589e+03 +711.77251919j
  -2.29146292e+03 +869.8489139j  -2.16221235e+03+1422.69809849j]
 [-2.71012410e+02 -471.7851572j   1.40469298e+03+2040.4276964j
   8.33124893e+02 +363.99146167j ... -2.87366993e+02-2200.98469621j
  -1.42141714e+03+1717.55571566j -9.57858690e+02 +152.0731563j ]
 [-9.95779487e+02 +272.61745955j  1.40449566e+03-1000.96338998j
  -3.57866896e+02 -593.07631485j ...  8.39213917e+02 +101.09339461j
   1.32748259e+03   +8.56570128j -1.39884623e+03 -551.41395527j]
 ...
 [-9.53432082e+02+1086.36198312j -1.23648027e+03 -829.55040286j
   9.66960172e+02 -401.33140115j ... -1.60061975e+03 -762.6853023j
  -7.07949970e+02+1544.16394705j -1.73715086e+02-1690.0932357j ]
 [-9.95779487e+02 -272.61745955j -1.39884623e+03 +551.41395527j
   1.32748259e+03   -8.56570128j ...  9.73067334e+02 -283.80483881j
  -3.57866896e+02 +593.07631485j  1.40449566e+03+1000.96338999j]
 [-2.7101241

==5474== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


Scipy RFFT

In [9]:
fft_type_conversion = random_matr + '''\
from scipy.fftpack import fft2, ifft2
expected = fft2(matr)
print(expected)
'''

print(memory_graph('scipy', fft_type_conversion))

==5577== Massif, a heap profiler
==5577== Copyright (C) 2003-2017, and GNU GPL'd, by Nicholas Nethercote
==5577== Using Valgrind-3.19.0 and LibVEX; rerun with -h for copyright info
==5577== Command: /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import\ numpy\ as\ np;n\ =\ 4096;m\ =\ 2\ *\ n;matr\ =\ np.zeros((n,\ m),\ dtype=np.complex64);matr\ =\ np.random.rand(n,\ m);from\ scipy.fftpack\ import\ fft2,\ ifft2;expected\ =\ fft2(matr);print(expected);
==5577== 


[[ 1.67734116e+07   -0.j          1.72816552e+03 -208.21451422j
  -1.63051005e+03+1899.92374546j ...  1.31693298e+03 +888.85095118j
  -1.63051005e+03-1899.92374546j  1.72816552e+03 +208.21451422j]
 [ 9.30648706e+02-2017.09511528j  5.81841633e+02+1216.02035828j
  -4.44948798e+02-2097.96165354j ... -1.38287987e+03 -162.24758065j
  -1.13602927e+03+1576.09569553j  5.12906474e+02+1026.6533001j ]
 [-1.48626998e+03 -571.86555711j -7.38151772e+01 +456.89198342j
  -1.53992422e+03 -531.72427685j ...  2.49089197e+01+1325.43672494j
  -3.25348312e+01 +399.77201062j -1.14433475e+03 -519.88864257j]
 ...
 [-1.88380674e+03-2615.78101278j  2.18987069e+03 -472.66367612j
   9.09113530e+02 +114.81163664j ...  1.51479263e+03-1741.53686326j
  -8.98902438e+02+1066.41420405j -1.07257998e+03 -498.97343632j]
 [-1.48626998e+03 +571.86555711j -1.14433475e+03 +519.88864257j
  -3.25348312e+01 -399.77201062j ... -3.47125534e+01 -278.54181938j
  -1.53992422e+03 +531.72427685j -7.38151772e+01 -456.89198342j]
 [ 9.30648

==5577== 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").
